Imports

In [41]:
import os
import json
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
import datetime

Utils

In [42]:
def translate(text, source='pl', target='en'):
    return GoogleTranslator(source, target).translate(text)

In [43]:
def extract(ancestor, selector=None, attribute=None, multiple=False):
    if selector:
        if multiple:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try: 
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()

In [48]:
selectors={
    'opinion_id':(None, "data-entry-id"),
    'author':( 'span.user-post__author-name',),
    'stars':('span.user-post__score-count',),
    'content_pl':('div.user-post__text',),
    'recommend' :('span.user-post__author-recomendation > em',),
    'pros':("div.review-feature__title--positives", None,True),
    'cons':('div.review-feature__title--negatives', None, True),
    'up_votes':('button.vote-yes','data-total-vote'),
    'down_votes':('button.vote-no','data-total-vote'),
    'purchased' :('span.user-post__published > time:nth-child(2)','datetime'),
    'published':('span.user-post__published > time:nth-child(1)','datetime'),
}

1. Send reqest for accessing first webpage with opinious about progect

In [45]:
with open('./cookie.json', 'r', encoding='UTF-8') as jf:
    headers= json.load(jf)

In [50]:
product_id = "107530595"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
while url:
    respomse = requests.get(url, headers=headers)
    if respomse.status_code ==200:
        page_dom = BeautifulSoup(respomse.text, 'html.parser')
        opinions= page_dom.select("div.js_product-review:not(.user-post--highlight)")
        all_opinion=[]
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *values)
                for key, values in selectors.items()
            }
            single_opinion['content_en']=translate(single_opinion['content_pl'])
            single_opinion['pros_en']=[translate(pros) for pros in single_opinion['content_pl']]
            single_opinion['cons_en']=[translate(cons) for cons in single_opinion['content_pl']]
            single_opinion['recommend']=True if single_opinion['recommend']=="Polecam" else False if single_opinion['recommend']==" Nie polecam" else None
            # single_opinion['stars'] = float(single_opinion('stars').split("/")[0].replace(",","."))
            # single_opinion['published'] = datetime.datetime.fromisoformat(single_opinion['published'])
            # single_opinion['purchased'] = datetime.datetime.fromisoformat(single_opinion['purchased']) if  single_opinion['purchased'] else None
            single_opinion['up_votes'] = int(single_opinion['up_votes'])
            single_opinion['down_votes'] = int(single_opinion['down_votes'])
            all_opinion.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+ page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None

print(all_opinion)
      

KeyboardInterrupt: 

In [ ]:
for single_opinion in all_opinion:
    try:
        single_opinion['published'] = single_opinion['published'].strftime("%Y-%m-%d, %H:%M:%S") if single_opinion['published'] else None
    except AttributeError:
        pass
    try:
        single_opinion['purchased'] = single_opinion['purchased'].strftime("%Y-%m-%d, %H:%M:%S") if single_opinion['purchased'] else None
    except AttributeError:
        pass
    


DB

In [39]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [41]:
with open(f"./opinions{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinion, jf, indent=4, ensure_ascii=False)